In [30]:
import json
import pandas as pd
from speed_estimation.paths import cars_path
from sklearn.metrics import mean_squared_error, mean_absolute_error
import plotly.express as px
import numpy as np

In [2]:
avg_speeds = []
with open("logs/run_log_a33e555cdc.log", "r") as fp:
    for line in fp:
        line_dict = json.loads(line[10:])
        if "avgSpeedLastMinute" in line_dict:
            avg_speeds.append(line_dict)

estimation = pd.DataFrame(avg_speeds)
estimation.head()
#df.to_csv("log_avg.csv", index=False)

,frameId,avgSpeedLastMinute
0,3000,69.19
1,6000,78.04
2,9000,82.83
3,12000,86.40
4,15000,70.70


In [23]:
cars = pd.read_csv(cars_path + "cars.csv")
estimation['avgSpeed'] = pd.to_numeric(estimation['avgSpeedLastMinute'], errors='coerce')

def avg_speed_for_time_ground_truth(timeStart, timeEnd):
    cars_to_avg = cars.loc[cars['start'].gt(timeStart) & cars['end'].le(timeEnd)]
    return cars_to_avg['speed'].mean()

def avg_speed_for_time_estimation(timeStart, timeEnd):
    timeStart *= 50
    timeEnd *= 50
    estimation_avg = estimation.loc[estimation['frameId'].gt(timeStart) & estimation['frameId'].le(timeEnd)]
    return estimation_avg['avgSpeed'].mean()

In [58]:
truth = []
predicted = []
timestamps = []
# 54min * 60s = 3.240s
for start in range(90, 3240, 60):
    end = start + 60
    truth.append(avg_speed_for_time_ground_truth(start, end))
    predicted.append(avg_speed_for_time_estimation(start,end))
    timestamps.append(end)

print("Truth vs. Predicted 50min divided in 1min slots: ")
print(f"mean_squared_error: {mean_squared_error(truth, predicted)}")
print(f"mean_absolute_error: {mean_absolute_error(truth, predicted)}")


Truth vs. Predicted 50min divided in 1min slots: 
mean_squared_error: 181.58177093796226
mean_absolute_error: 9.643268942027948


In [61]:
mean_error = pd.DataFrame(dict(timestamps=timestamps, 
                                mean_absolute_error=np.absolute(np.array(truth) - np.array(predicted)),
                                mean_error=np.array(truth) - np.array(predicted)))

In [62]:
fig = px.line(mean_error, x="timestamps", y="mean_absolute_error", title='Mean absolute error')
fig.show()

In [64]:
fig = px.line(mean_error, x="timestamps", y="mean_error", title='Mean error (truth - estimation)')
fig.show()